In [1]:
import pandas as pd
import numpy as np

In [3]:
#Using matches_final.csv and merged_lineups_2022.csv
mfn = pd.read_csv('../../Data/Data Preparation/matches_final.csv')
lineup = pd.read_csv('../../Data/Data Preparation/merged_lineups_final.csv')
ppt = 11 #Player per team
statsCount = 36 #Number of stats used
lineup.head(5)

,team_id,fixture_id,player_id,player_name,number,position,date,club,lastname,firstname,...,FKAcc.,Penalties,Volleys,GKPositioning,GKDiving,GKHandling,GKKicking,GKReflexes,Overall,Potential
0,1,2864,1251,Adrian,13,G,2016-01-02,West Ham United,Adrian,Adrian,...,17,44,12,76,78,76,71,85,78,78
1,1,2864,1261,Aaron Cresswell,3,D,2016-01-02,West Ham United,Cresswell,Aaron,...,69,59,29,9,14,7,9,12,73,75
2,1,2864,336,James Tomkins,5,D,2016-01-02,West Ham United,Tomkins,James,...,31,43,28,9,11,8,6,14,73,74
3,1,2864,538,James Collins,19,D,2016-01-02,West Ham United,Collins,James,...,64,51,36,15,15,12,10,11,74,74
4,1,2864,1416,Angelo Ogbonna,21,D,2016-01-02,West Ham United,Ogbonna,Angelo,...,36,39,30,4,5,3,5,4,79,79


In [4]:
mfn.head(5)

,fixture_id,date,home_id,home_name,home_scored,home_logo,away_id,away_name,away_scored,away_logo
0,18243482,2011-08-13,2,Blackburn Rovers,1,https://cdn.sportmonks.com/images/soccer/teams...,29,Wolverhampton Wanderers,2,https://cdn.sportmonks.com/images/soccer/teams...
1,18243483,2011-08-13,11,Fulham,0,https://cdn.sportmonks.com/images/soccer/teams...,15,Aston Villa,0,https://cdn.sportmonks.com/images/soccer/teams...
2,18243484,2011-08-13,8,Liverpool,1,https://cdn.sportmonks.com/images/soccer/teams...,3,Sunderland,1,https://cdn.sportmonks.com/images/soccer/teams...
3,18243485,2011-08-13,47,Queens Park Rangers,0,https://cdn.sportmonks.com/images/soccer/teams...,16,Bolton Wanderers,4,https://cdn.sportmonks.com/images/soccer/teams...
4,18243486,2011-08-13,17,Wigan Athletic,1,https://cdn.sportmonks.com/images/soccer/teams...,33,Norwich City,1,https://cdn.sportmonks.com/images/soccer/teams...


In [5]:
numsLnup = ['team_id', 'fixture_id', 'Height', 'Weight', 'BallControl', 'Marking', 'Dribbling', 'SlideTackle',
 'StandTackle', 'Aggression', 'Reactions', 'Att.Position', 'Interceptions', 'Vision', 'Crossing', 'ShortPass', 'LongPass',
 'Acceleration', 'Stamina', 'Strength', 'Balance', 'SprintSpeed', 'Agility', 'Jumping', 'Heading', 'ShotPower', 'Finishing',
 'LongShots', 'Curve', 'FKAcc.', 'Penalties', 'Volleys', 'GKPositioning', 'GKDiving', 'GKHandling', 'GKKicking',
 'GKReflexes', 'Overall']
numsCols = lineup[numsLnup]
colsName = ['id','home_id','away_id']
for i in range(statsCount):
    colsName.append('gap_stat_' + str(i+1))

iptStats = ['Vision','LongPass','SprintSpeed','Acceleration','Finishing' ,'LongShots', 'Curve']
for i in iptStats:
    colsName.append('gap_max_'+ i)

FormStats = pd.DataFrame(columns=colsName)

for i in range(int(lineup.shape[0]/(2*ppt))):
    sum = 0
    for j in range(2*ppt):
        if numsCols['team_id'][i*22+ j] == mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22+ j]]['home_id'].values[0]:
            sum = np.array(sum + numsCols.iloc[i*22+ j, 2:].values)
        else: 
            sum = np.array(sum - numsCols.iloc[i*22+ j, 2:].values)
    sum = sum/11

    for j in iptStats:
        gapMaxStat = 0
        for k in range(2):
            if numsCols['team_id'][i*22+ k*11] == mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['home_id'].values[0]:
                gapMaxStat = gapMaxStat + numsCols[j].iloc[i*22+ k*11: i*22+ (k+1)*11].max()
            else:
                gapMaxStat = gapMaxStat - numsCols[j].iloc[i*22+ k*11: i*22+ (k+1)*11].max()
        sum = np.append(sum, gapMaxStat)
    match_id = int(numsCols.fixture_id.iloc[i*22])
    home_id = int(mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['home_id'].values[0])
    away_id = int(mfn[mfn['fixture_id'] == numsCols.fixture_id.iloc[i*22]]['away_id'].values[0])
        
    ids = np.array([match_id, home_id, away_id])
    sum = np.concatenate((ids, sum), axis= None)
    FormStats.loc[i] = sum
FormStats[['id','home_id','away_id']] = FormStats[['id','home_id','away_id']].astype(int)
FormStats


,id,home_id,away_id,gap_stat_1,gap_stat_2,gap_stat_3,gap_stat_4,gap_stat_5,gap_stat_6,gap_stat_7,...,gap_stat_34,gap_stat_35,gap_stat_36,gap_max_Vision,gap_max_LongPass,gap_max_SprintSpeed,gap_max_Acceleration,gap_max_Finishing,gap_max_LongShots,gap_max_Curve
0,2864,1,8,2.090909,-0.363636,-6.000000,0.000000,-7.909091,-1.363636,-3.090909,...,0.363636,1.545455,-3.272727,-9.0,-1.0,5.0,3.0,-5.0,0.0,-10.0
1,2876,3,15,-2.181818,-2.636364,-0.727273,-5.363636,0.636364,-5.454545,-7.363636,...,0.000000,1.363636,-0.090909,-3.0,5.0,10.0,10.0,10.0,8.0,0.0
2,2886,42,52,-1.272727,-1.545455,-0.272727,-5.181818,-2.636364,-1.454545,-1.545455,...,1.454545,1.636364,2.454545,-2.0,1.0,1.0,3.0,5.0,4.0,17.0
3,2897,19,20,0.727273,-3.272727,3.181818,-2.363636,-0.363636,-1.545455,-1.909091,...,-1.909091,1.000000,5.727273,16.0,7.0,11.0,16.0,1.0,7.0,5.0
4,2908,33,65,-1.727273,-5.454545,-5.636364,-12.000000,-6.272727,-4.909091,-5.727273,...,-0.363636,0.272727,-3.818182,-2.0,-6.0,0.0,5.0,4.0,5.0,-6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,18165753,3457,83,-0.818182,-2.090909,-7.363636,1.636364,-7.909091,-0.454545,2.181818,...,-1.818182,-1.272727,-7.454545,-1.0,-5.0,-8.0,-8.0,-4.0,1.0,1.0
11427,18165757,377,214,-3.454545,0.363636,-1.636364,-1.909091,-3.636364,-4.454545,-5.818182,...,-0.090909,1.727273,-1.818182,-6.0,-6.0,0.0,4.0,-4.0,-9.0,-5.0
11428,18165743,594,485,2.272727,1.454545,0.636364,3.545455,0.000000,0.545455,0.454545,...,-1.727273,0.090909,0.272727,1.0,-4.0,-2.0,-6.0,0.0,-6.0,-3.0
11429,302049,613,2930,-0.272727,-0.909091,-4.454545,-6.818182,-3.000000,-5.363636,-6.727273,...,0.636364,-1.090909,-4.545455,-8.0,-7.0,3.0,10.0,-3.0,-6.0,-3.0


In [6]:
FormStats.to_csv('../../Data/Final Data Modelling/FormStatsDone.csv')